# Minimum Complementary Energy

Over the past two weeks, we have explored the displacement resulting from distributed pressure and the Hertz contact solution. Now, we shift our focus to real contact problems, emphasizing **Variational Principles** and the **Minimum Complementary Energy** approach.

![Contact model discussed in first step](figures/Problem_mapping.png)

Here we simplify our model as the contact between a rigid rough surface and an elastic flat surface[4].

![rigid rough surface elastic contact](figures/rigid_rough_surface_elastic_contact.jpg)


We want to know pressure distribution $P(x,y)$, and we define our gap function as:

$$
g(x, y)=u_z(x, y)-h(x, y)
$$

where $u_z(x, y)$ is the normal displacement, the separation of two surfaces $h(x,y)$ can sometimes be $h(r)=-\frac{r^2}{2 R}$, like Rey(2017)[5].

As the first chapiter of Lucas(2020)[1], the potential energy can be derived from elastic energy relationship $\frac{1}{2} \int_v \varepsilon: \mathbb{C}: \varepsilon d v$ to:

$$
E_p\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s
$$

If we minimize this potential energy with respect to $u_z$, then we call it **Primal minimization problem**, we will end up with a displacement solution.

$$
\min _{g\left[u_z\right] \geqslant 0} E_p\left(u_z, p\left[u_z\right]\right)
$$

Here we focus on complementary energy minimization, called **Dual minimization problem**, where complementary energy is defined as:

$$
E_c\left(u_z, p\right)=\frac{1}{2} \int_s u_z \cdot p d s-\int_s p h d s
$$

Then the minimization could be:

$$
\min _{p \geqslant 0} E_c\left(u_z[p], p\right)
$$

It is obvious that we will end up with a pressure solution in this procedure.



## Q1: Legendre transform for dual optimization?

From what we discuss above, we have two constraints, 

$$
\frac{1}{S} \int_s p d s=\bar{p}
$$

And

$$
p \geq 0
$$

To minimize the complementary energy, we consider:

$$
\lim _{\varepsilon \rightarrow 0} \frac{E_c(p+\varepsilon v)-E_c(p)}{\varepsilon}
$$

Here we can derive a gap function by:

$$
\frac{\partial E_c}{\partial p}=u_z[p]-h=g
$$



We choose the same background as a start point to verify this energy minimization, and define $ \bar{p}=\frac{F}{S} $

In [26]:
import numpy as np
from scipy.optimize import minimize

#define the parameters
F = 1e2  # total force
R = 1  # radius of demi-sphere
L = 2  # domain size
S = L**2  # domain area
p_bar = F / S  # average pressure


#define material parameters
E = 1e3  # Young's modulus
nu = 0.3  # Poisson's ratio
E_star = E / (1 - nu**2)  # plane strain modulus 


## Q2 How can we find h? By finding gap function?

In [27]:
#We generate a 2D coordinate space
n = 100
m = 100

x, y = np.meshgrid(np.linspace(0, L, n, endpoint=False), np.linspace(0, L, m, endpoint=False))#notice here that we use endpoint=False to avoid having the last point

x0 = 1
y0 = 1


In [28]:
# Define the separation h          #Since we are using the same model, we can use the same h function
def h(x, y, R):
    r = np.sqrt(x**2 + y**2)
    return -r**2 / (2 * R)

we give our analytical normal displacement solution by Hertz

$$
\bar{u}_z=\frac{1-\nu^2}{E} \frac{\pi p_0}{4 a}\left(2 a^2-r^2\right), \quad r \leqslant a
$$


## Q3 confused about my implementation, with hertz solution? that is not the case!

In [29]:
# Define u_z based on the given formula
def u_z(r, E, nu, p_bar, a):
    return (1 - nu**2) / E * (np.pi * p_0 / 4 * a) * (2 * a**2 - r**2)

# Define the complementary energy function
def E_c(p, x, y, R=1):
    # Calculate the gap
    gap = u_z - h(x, y, R)
    return 0.5 * np.sum(u_z * p) - np.sum(p * gap)

## Q4: not understand too much the following

In [30]:
# Objective function for minimization
def objective(p, x, y):
    return E_c(p, x, y)

# Constraint: average pressure
def constraint_avg_pressure(p):
    return np.sum(p) / S - p_bar

# Constraints
cons = [{'type': 'eq', 'fun': constraint_avg_pressure}]

In [31]:

# Perform the minimization
result = minimize(lambda p: objective(p, xx, yy), p, method='SLSQP', constraints=cons, options={'disp': True})


NameError: name 'p' is not defined

## Reference:

[1] Frérot, Lucas Henri Galilée. ‘Bridging Scales in Wear Modeling with Volume Integral Methods for Elastic-Plastic Contact’, n.d.

[2] Yastrebov, Vladislav A. Numerical Methods in Contact Mechanics. Numerical Methods in Engineering Series. Hoboken, NJ: Wiley, 2013.

[3] Kalker, J. J. ‘Variational Principles of Contact Elastostatics’. IMA Journal of Applied Mathematics 20, no. 2 (1977): 199–219. https://doi.org/10.1093/imamat/20.2.199.

[4] Vladislav A. Yastrebov, Contact mechanics and elements of tribology, Lecture 4.b, Contact and transport at small scales, Open Course Contact Mechanics and Elements of Tribology, January 23, 2024, https://cmet.yastrebov.fr/index.html

[5] Rey, Valentine, Guillaume Anciaux, and Jean-François Molinari. ‘Normal Adhesive Contact on Rough Surfaces: Efficient Algorithm for FFT-Based BEM Resolution’. Computational Mechanics 60, no. 1 (July 2017): 69–81. https://doi.org/10.1007/s00466-017-1392-5.

